In [1]:
from helper import (
    hash256,
    int_to_little_endian,
    little_endian_to_int,
    read_varint,
    encode_varint
)
from io import BytesIO
import json
import requests
from script import Script

# Transaction class (version, inputs, outputs, locktime)
class Tx:
    
    def __init__(self, version, tx_ins, tx_outs, locktime, testnet=False):
        self.version = version
        self.tx_ins = tx_ins
        self.tx_outs = tx_outs
        self.locktime = locktime
        self.testnet = testnet
        
    def __repr__(self):
        tx_ins = ''
        for tx_in in self.tx_ins:
            tx_ins += tx_in.__repr__() + '\n'
            
        tx_outs = ''
        for tx_out in self.tx_outs:
            tx_outs += tx_out.__repr__() + '\n'
        
        return f'tx : {self.id()}\nversion : {self.version}\ntx inputs : \n{tx_ins}tx outputs : \n{tx_outs}locktime : self.locktime'
    
    def id(self):
        '''Transaction hash in hexadecimal'''
        return self.hash().hex()
    
    def hash(self):
        '''Binary hash of the legacy serialization'''
        return hash256(self.serialize())[::-1]
    
    @classmethod
    def parse(cls, s, testnet=False):
        version = little_endian_to_int(s.read(4))
        
        n_inputs = read_varint(s)
        inputs = []
        for _ in range(n_inputs):
            inputs.append(TxInput.parse(s))
        
        n_outputs = read_varint(s)
        outputs = []
        for _ in range(n_outputs):
            outputs.append(TxOutput.parse(s))
            
        locktime = little_endian_to_int(s.read(4))
        return cls(version, inputs, outputs, locktime, testnet=testnet)
        
    def serialize(self):
        '''Returns the byte serialize of the Tx'''
        result = int_to_little_endian(self.version, 4)
        result += encode_varint(len(self.tx_ins))
        for tx_in in tx_ins:
            result += tx_in.serialize()
        result += encode_varint(len(self.tx_outs))
        for tx_out in tx_outs:
            result += tx_out.serialize()
        result += int_to_little_endian(self.locktime, 4)
        return result
    
    def fee(self, testnet=False):
        '''Calculate fee'''
        total_in_value = 0
        for tx_in in self.tx_ins:
            total_in_value += tx_in.value(testnet=False)
            
        total_out_value = 0
        for tx_out in self.tx_outs:
            total_out_value += tx_out.amount
            
        return total_in_value - total_out_value
    
        
# Transaction input class        
class TxInput:
    
    def __init__(self, prev_tx, prev_index, script_sig=None, sequence=0xffffffff):
        self.prev_tx = prev_tx
        self.prev_index = prev_index
        if script_sig is None:
            self.script_sig = Script()
        else:
            self.script_sig = script_sig
        self.sequence = sequence
        
    def __repr__(self):
        return f'{self.prev_tx.hex()}:{self.prev_index}'
        
    @classmethod
    def parse(cls, s):
#         prev_tx = little_endian_to_int(s.read(32)) # need to check
        prev_tx = s.read(32)[::-1] # in bytes?
        prev_index = little_endian_to_int(s.read(4))
#         len_script_sig = read_varint(s)
#         script_sig = little_endian_to_int(s.read(len_script_sig))
        script_sig = Script.parse(s)
        sequence = little_endian_to_int(s.read(4))
        return cls(prev_tx, prev_index, script_sig, sequence)
    
    def serialize(self):
        '''Returns the byte of serialization of Tx input'''
        result = self.prev_tx[::-1]
        result += int_to_little_endian(self.index, 4)
        result += self.script_sig.serialize()
        result += int_to_little_endian(self.sequence, 4)
        return result
    
    def fetch_tx(self, testnet=False):
        return TxFetcher.fetch(self.prev_tx.hex(), testnet)
    
    def value(self, testnet=False):
        tx = self.fetch_tx(testnet=testnet)
        return tx.tx_outs[self.prev_index].amount
    
    def get_script_verify(self, testnet=False):
        tx = self.fetch_tx(testnet=testnet)
        return tx.tx_outs[self.prev_index].script_verify
    
    
# Transacrion output class
class TxOutput:
    
    def __init__(self, amount, script_verify):
        self.amount = amount
        self.script_verify = script_verify
        
    def __repr__(self):
        return f'{self.amount}(in satosi):{self.script_verify}'
    
    @classmethod
    def parse(cls, s):
        amount = little_endian_to_int(s.read(8))
        script_verify = Script.parse(s)
        return cls(amount, script_verify)
    
    def serialize(self):
        '''Returns the byte serialization of the Tx output'''
        result = int_to_little_endian(self.amount, 8)
        result += self.script_verify.serialize() # script class?
        return result
    
# Tx Fetcher class
class TxFetcher:
    
    cache = {}
    
    @classmethod
    def get_url(cls, testnet=False):
        if testnet:
            return 'http://testnet.programmingbitcoin.com'
        else:
            return 'http://mainnet.programmingbitcoin.com'
        
    @classmethod
    def fetch(cls, tx_id, testnet=False, fresh=False):
        if fresh or (tx_id not in cls.cache): # need to fetch
            url = f'{cls.get_url(testnet)}/tx/{tx_id}.hex'
            response = requests.get(url)
            try:
                raw = bytes.fromhex(response.text.strip())
            except ValueError:
                raise ValueError(f'unexpected response :{response.text}')
            if raw[4] == 0: # raw[4] == 0 은 어떤 케이스?
                raw = raw[:4] + raw[6:]
                tx = Tx.parse(BytesIO(raw), testnet=testnet)
                tx.locktime = little_endian_to_int(raw[-4:])
            else:
                tx = Tx.parse(BytesIO(raw), testnet=testnet)
            if tx.id() != tx_id:
                raise ValueError(f'not the same id: {tx.id()} vs {tx_id}')
                
            cls.cache[tx_id] = tx
        cls.cache[tx_id].testnet = testnet
        return cls.cache[tx_id]